### Unification Algorithm
First realize some basic discriminance.

In [11]:
from sympy import symbols, Function
from sympy.core.function import AppliedUndef
from sympy.core.symbol import Symbol

def is_term(x):
    return x.is_symbol or isinstance(x, AppliedUndef)

def is_var(x):
    return x.is_symbol and str(x).islower()

def is_const(x):
    return x.is_symbol and not str(x).islower()

def is_comp(x):
    return isinstance(x, AppliedUndef)

def is_op(x):
    return isinstance(x, Function)

Try to substitute variables to unify two expressions, making them the same.

In [12]:
def unify(x, y, theta=None):
    if theta is None:
        theta = {}
    if theta is None:
        return None
    elif x == y:
        return theta
    elif is_var(x):
        return unify_var(x, y, theta)
    elif is_var(y):
        return unify_var(y, x, theta)
    elif is_comp(x) and is_comp(y):
        if x.func == y.func and len(x.args) == len(y.args):
            for arg_x, arg_y in zip(x.args, y.args):
                theta = unify(arg_x, arg_y, theta)
                if theta is None:
                    return None
            return theta
    return None

def unify_var(var, x, theta):
    if var in theta:
        return unify(theta[var], x, theta)
    elif x in theta:
        return unify(var, theta[x], theta)
    elif occur_check(var, x, theta):
        return None
    else:
        theta[var] = x
        return theta

def occur_check(var, x, theta):
    if var == x:
        return True
    elif is_variable(x) and x in theta:
        return occur_check(var, theta[x], theta)
    elif isinstance(x, AppliedUndef):
        return any(occur_check(var, arg, theta) for arg in x.args)
    return False

x, y = symbols('x y')
John, Jane, Bill, Elizabeth = symbols('John Jane Bill Elizabeth')
Mother = Function('Mother')
Knows = Function('Knows')

expr1 = Knows(John, x)
expr2 = Knows(John, Jane)
expr3 = Knows(y, Bill)
expr4 = Knows(y, Mother(y))
expr5 = Knows(y, Elizabeth)

print("UNIFY(Knows(John, x), Knows(John, Jane)) =", unify(expr1, expr2))
print("UNIFY(Knows(John, x), Knows(y, Bill)) =", unify(expr1, expr3))
print("UNIFY(Knows(John, x), Knows(y, Mother(y))) =", unify(expr1, expr4))
print("UNIFY(Knows(John, x), Knows(y, Elizabeth)) =", unify(expr1, expr5))

UNIFY(Knows(John, x), Knows(John, Jane)) = {x: Jane}
UNIFY(Knows(John, x), Knows(y, Bill)) = {y: John, x: Bill}
UNIFY(Knows(John, x), Knows(y, Mother(y))) = {y: John, x: Mother(y)}
UNIFY(Knows(John, x), Knows(y, Elizabeth)) = {y: John, x: Elizabeth}
